<a href="https://colab.research.google.com/github/kcarnold/cs344/blob/main/Example_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try: import fastbook
except ImportError: import subprocess; subprocess.run(['pip','install','-Uq','fastbook'])

# Lab 1

We're going to start with the cat-vs-dog classifier that we built in chapter 1 and try changing a few things.

Before we get started, discuss with your partner how you think each one of these changes would change the performance of the classifier. Write your answers below.

> Train the model on less data

It will be less accurate due to less base data for the AI to work with.

> Use an architecture with fewer layers

It will be less accurate.

> Have it predict the breed of pet instead of just cat/dog.

It is trying to determine different things. One is classication (cat/dog) and the other is regression (breed of dog).

Now let's start. You might see a task like this:

> Train a classifier to distinguish between images of cats and dogs.
> 
> * Use the [Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/).
> * Fine-tune a 34-layer ResNet model for 1 epoch
> * Report the error rate on a held-out validation set of 20% of the data.

The first code block from chapter 1 accomplishes this task. Retype or copy-paste it here, but add some comments if you haven't already:

In [2]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.157933,0.032194,0.008796,01:05


epoch,train_loss,valid_loss,error_rate,time
0,0.056920,0.047423,0.011502,01:17


**Question**: Use `learn.fine_tune?` to bring up the documentation. Based on that:

1. What does the "1" parameter to `fine_tune` do?
2. What do you think the two different progress bars probably represent?

**Question**: Give an example filename for one of the images in the dataset. Does the dataset label it as a cat or a dog? How can you tell?

In [3]:
#Abc.jpg would be labeled as a cat because it starts with a capital letter.

**Question**: How many images were in the training set? Validation set? (How can you easily tell?)

In [4]:
#80% was in the training set and 20% was in the validation set.

You can tell by the parameter valid_pct = 0.2.

**Question**: About how many of those images were classified correctly?

7390 - (7390 * error%) = 7315 images were classified correctly, or about 98.9%.



**Question**: What is the output of the model?

1. whether the image contains one cat (`True`) or many cats (`False`)

*2. whether the image contains a cat (`True`) or a dog (`False`)*

3. what breed of cat or dog is in the image (`British_Shorthair`, `chihuahua`, etc.)
4. the name of the animal in the image (`Fido`, `Whiskers`, etc.)

**Exercise**: Use an 18-layer ResNet instead (`resnet18`). How does the error rate compare?

In [5]:
#The error rate is smaller with resnet18 (0.010149 to 0.008796).

Consider the We just changed the *architecture*, but not anything else. We could also change:

* The *data*: give it different images (or more or fewer of the same images)
* The *task*: have it try to predict something different
* The *hyperparameters*: train it a different way, e.g., let it train longer (more epochs) or adjust itself faster (higher learning rate)

Let's change the data.

**Exercise**: Hold out 90% (instead of 20%) of the data for validation. (How many images will the training set have now?) How does the accuracy compare?

(*Think*: Should you use the resnet18 or the resnet34 model here? What are the pros and cons?)

In [6]:
#10% in the training set and 90% in the validation set. 
#The accuracy went down with an error rate of 0.031123 compared to 0.010149.

**Question**: How might we change the *task*? That is, what different things could we have the model output?

Think about that for a moment.

.

.

.

The model's outputs are (generally) determined by what sort of *labels* we have for our data. Let's have a look at what the files are named. We could open up our data directory, or we could look in Python.

In [7]:
[o.name for o in get_image_files(path)][:10] # show only the first 10

['Birman_10.jpg',
 'Maine_Coon_270.jpg',
 'pomeranian_123.jpg',
 'havanese_158.jpg',
 'Egyptian_Mau_121.jpg',
 'havanese_56.jpg',
 'chihuahua_198.jpg',
 'wheaten_terrier_35.jpg',
 'boxer_22.jpg',
 'staffordshire_bull_terrier_202.jpg']

In [8]:
# Aside: fastai added a nice trick that makes this easy:
get_image_files(path).attrgot('name')

(#7390) ['Birman_10.jpg','Maine_Coon_270.jpg','pomeranian_123.jpg','havanese_158.jpg','Egyptian_Mau_121.jpg','havanese_56.jpg','chihuahua_198.jpg','wheaten_terrier_35.jpg','boxer_22.jpg','staffordshire_bull_terrier_202.jpg'...]

So it looks like we should be able to get the *breed* pretty easily. All we need to do is strip off everything after the last underscore. (Is this exercise familiar?)

These names are passed to the `name_func`. So all we need to do is strip off the number and file extension at the end to get the breed.

**Exercise**: Write a function called `get_breed` that takes a string and strips off everything after the last underscore.

In [9]:
def get_breed(file_name):
    return file_name[:file_name.rfind('_')]

Now let's test that function.

In [10]:
# When a Python expression gets complicated, I like to spread it onto multiple lines.
# Often an expression is already inside some parentheses so you can just press Enter,
# but for "top-level" expressions we need this trick of surrounding the whole thing in parens.
# Aside from the attrot and map, this is just standard Python here, no special magic.
(
    get_image_files(path)
    .attrgot('name')
    .map(get_breed)
)

(#7390) ['Birman','Maine_Coon','pomeranian','havanese','Egyptian_Mau','havanese','chihuahua','wheaten_terrier','boxer','staffordshire_bull_terrier'...]

Let's check how many of each breed we have.

In [11]:
Counter(
    get_image_files(path)
    .attrgot('name')
    .map(get_breed)
)

Counter({'Abyssinian': 200,
         'Bengal': 200,
         'Birman': 200,
         'Bombay': 200,
         'British_Shorthair': 200,
         'Egyptian_Mau': 200,
         'Maine_Coon': 200,
         'Persian': 200,
         'Ragdoll': 200,
         'Russian_Blue': 200,
         'Siamese': 200,
         'Sphynx': 200,
         'american_bulldog': 200,
         'american_pit_bull_terrier': 200,
         'basset_hound': 200,
         'beagle': 200,
         'boxer': 200,
         'chihuahua': 200,
         'english_cocker_spaniel': 200,
         'english_setter': 200,
         'german_shorthaired': 200,
         'great_pyrenees': 200,
         'havanese': 200,
         'japanese_chin': 200,
         'keeshond': 200,
         'leonberger': 200,
         'miniature_pinscher': 200,
         'newfoundland': 200,
         'pomeranian': 200,
         'pug': 200,
         'saint_bernard': 200,
         'samoyed': 200,
         'scottish_terrier': 199,
         'shiba_inu': 200,
         'staf

Now, let's change the model to output the breed instead of whether or not the thing is a cat.

**Predict**: Will the accuracy number go up or down? Why?

**Exercise**: Go back to the initial setup (ResNet-34 model and full dataset), but instead of predicting cat vs dog, now predict breed.

In [ ]:
#Our prediction is that the accuracy will plummet because 
#there are many more categories for it to distinguish between meaning chance for the error is larger.

from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def get_breed(x): return x[:x.rfind('_')]

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=get_breed, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,1.493640,0.307790,0.104195,01:04


epoch,train_loss,valid_loss,error_rate,time


**Reflect**: What accuracy do you get with this classifier? Is it better or worse? How can you tell?

-The error rate seems to be worse? (0.105548 / 0.089986) 

# Open-ended!

If you got through that quickly, now try one or two more experiments of your own choice. For example, you might try changing a hyperparameter (epochs, learning rate, ...), try a different dataset size, try different random seeds, ...

# Summarize

1. Write a short summary of the results of all of the experiments you performed in this lab.
    - We learned that ResNet18 (18 layers) is more accurate than ResNet34 (34 layers), in this experiment. 
    - We learned that 80% training set and a 20% validation set appears to be optimal as opposed to 90% training set and 10% validation set. This shows the importance of the validation set as it can help correct mistakes made in training set. 
    - We learned that training an AI with many categories is less accurate than training an AI with fewer categories due to a greater opportunity for error, classification vs. regression.
    
2. Reflect on how confident you are that each of those comparisons didn't just happen by chance. Also reflect on which of the results would be likely to generalize to other tasks.
    - We are confident that these comparisons did not happen by coincidence because we re-ran the kernel and retrieved similar results.

# Wrap-up

* Make sure you Save this notebook.
* From the Kernel menu, select "Restart and Run All". This will take a few minutes, but will ensure that all of your code still runs. Double-check that your results still make sense.
* Commit and push. Check that your notebook is visible on GitHub.